## Part V: Training a Model with Data Programming

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ features snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

### Loading the `CandidateSet`, feature matrix, and label matrix

In [4]:
from snorkel.models import CandidateSet
train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()

In [5]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

%time F_train = feature_manager.load(session, train, 'Train Features')

CPU times: user 3min 14s, sys: 21.7 s, total: 3min 36s
Wall time: 3min 47s


In [6]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

%time L_train = label_manager.load(session, train, 'LF Labels')

CPU times: user 3.6 s, sys: 225 ms, total: 3.82 s
Wall time: 4.03 s


## Train Generative Model

In [7]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
# gen_model.train(L_train, n_iter=3000, rate=1e-5)
gen_model.train(L_train, n_iter=10000, rate=1e-2)

ImportError: cannot import name LogReg

In [ ]:
gen_model.save(session, 'Generative Params')

In [ ]:
train_marginals = gen_model.marginals(L_train)

## Train Discriminative Model

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=2000, rate=1e-4)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

## Assess Performance on Development Set

In [ ]:
from snorkel.models import CandidateSet
dev = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates').one()

### Loading the Development Feature Matrix

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_dev = feature_manager.load(session, dev, 'Train Features')

In [ ]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [ ]:
from snorkel.models import CandidateSet
from snorkel.annotations import LabelManager
label_manager = LabelManager()
L_dev = label_manager.load(session, dev, 'Hardware Development Labels -- Gold')

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [ ]:
F_dev.shape

### Calculating F1 Score

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

In [ ]:
from hardware_utils import part_error_analysis, table_info

In [ ]:
# for c in list(fp)[:5]:
#     print c
# c = list(fp)[10]
# error_analysis(c)
# c = list(fp)[10]
# part_error_analysis(c)
# print c.temp.parent.row_num

# for c in list(fp)[:5]:
#     print c
# print c.temp.parent.html_anc_tags

In [ ]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

In [ ]:
from pprint import pprint
FP_list = sorted(list(FP))
pprint(FP_list)

In [ ]:
feature_weights = disc_model.get_feature_weights(F_train, reverse=False)

In [ ]:
pprint(feature_weights[:25])

In [ ]:
# TODO: canonize some of these helper functions
# from snorkel.utils import get_keys_by_candidate

# def features_by_weight(candidate, feature_matrix, feature_weights):
#     feats = set(get_keys_by_candidate(feature_matrix, candidate))
#     return sorted([f_w for f_w in feature_weights if f_w[0] in feats], 
#                   key=lambda x:abs(x[1]), reverse=True)

In [ ]:
from hardware_utils import entity_to_candidates
from pprint import pprint

entity = sorted(list(FP))[6]
print entity
print

matches = entity_to_candidates(entity, fp)
candidate = matches[0]
print candidate
print

pprint(disc_model.get_candidate_feature_weights(
        candidate, F_dev, feature_weights=feature_weights))

In [ ]:
total = 0
for f,v in features_by_weight(candidate, F_dev, feature_weights):
    total += v
print total

In [ ]:
from snorkel.utils import get_keys_by_candidate

# F_train
for f in get_keys_by_candidate(F_dev, candidate)[:]: print f

In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ final');

In [ ]:
from snorkel.utils import get_keys_by_candidate
for a in sorted(get_keys_by_candidate(F_dev, sample)):
    print a

In [ ]:
from snorkel.lf_helpers import *
def LF_min_col(c):
    return -1 if 'min' in get_col_ngrams(c.temp) or 'min.' in get_col_ngrams(c.temp) else 0

LF_min_col(sample)

The End.